In [1]:
import os
path = os.path.join('/home/santiago/Documents/dev/reservoirpy')
import sys
sys.path.insert(0,path)

In [2]:
from reservoirpy.simulationpy import sim
from reservoirpy.pvtpy import black_oil as bl
from reservoirpy.wellpy import path as ph
from datetime import date
import pandas as pd
import numpy as np
import pyvista as pv
import vtk 
from shapely.geometry import Point
import math
import matplotlib.pyplot as plt

# Grid

In [3]:
grid=sim.grid( 
    grid_type='cartesian',
    nx = 10,
    ny = 10,
    nz = 5,
    dx = 100,
    dy = 100,
    dz=  50,
    origin = Point(100,100,-5000),
    petrophysics = {'PORO':0.18,'PERMX':200, 'PERMY':300,'PERMZ':100,'RT':0},
    azimuth=0,
    dip=0
)

# Phases

In [4]:
phase = ['water']

# Pvt

In [5]:
water = bl.water(formation='fm_1',pb=2000,salinity=13500, temp=60)
water.pvt_from_correlations()

# Wells

In [6]:
#Create the well object
name1 = 'well-1'
rte1 = 200 # Rotary table Elevation
surf_coord1 = [255,741]#Point(1000100,1000000,520)
crs1 = 'EPSG:3117'
tops1 = ph.tops({'formation':['fm1'],'md_top':[5200],'md_bottom':[5450]})
perf1 = ph.perforations({'md_top':[5300],'md_bottom':[5310]})
cons1 = {
    'date':np.array([np.datetime64('2020-08-21'),np.datetime64('2021-01-05')]),
    'constrain':['qw','bhp'],
    'value':[100,250]
}
td1 = 6000

w1 = ph.well(name=name1, 
             rte=rte1, 
             surf_coord=surf_coord1, 
             td = td1,
             tops=tops1,
             perforations = perf1,
             constrains=cons1,
             crs=crs1)


#Create the well object
name2 = 'well-2'
rte2 = 200 # Rotary table Elevation
surf_coord2 = [658,478]#Point(1000100,1000000,520)
crs2 = 'EPSG:3117'
tops2 = ph.tops({'formation':['fm1'],'md_top':[5200],'md_bottom':[5450]})
perf2 = ph.perforations({'md_top':[5300],'md_bottom':[5310]})
cons2 = {
    'date':np.array([np.datetime64('2020-08-21'),np.datetime64('2021-01-05')]),
    'constrain':['bhp','bhp'],
    'value':[100,250]
}
td2 = 6000

w2 = ph.well(name=name2, 
             rte=rte2, 
             surf_coord=surf_coord2, 
             td = td2,
             tops=tops2,
             perforations = perf2,
             constrains=cons2,
             crs=crs2)


wells = ph.wells_group(w1,w2)

divide by zero encountered in true_divide
invalid value encountered in multiply


In [7]:
wells.wells['well-1'].constrains

{'date': array(['2020-08-21', '2021-01-05'], dtype='datetime64[D]'),
 'constrain': ['qw', 'bhp'],
 'value': [100, 250]}

# Numerical

In [8]:
start_date = date(2020,8,21)
end_date = date(2022,8,21)

dates = pd.date_range(start_date,end_date,freq='M')
numerical = sim.numerical(
    relaxation = 1,
    max_iter=25,
    date_range = dates.values
)

# Initial Conditions

In [9]:
init_conds = sim.initial_conditions(
    pi = 2500,
    woc = -5200,
    cap_press_init = False
)

# Model

In [10]:
sim_model = sim.model(
      grid = grid,
      phase = phase,
      pvt = {'water':water},
      wells = wells,
      numerical = numerical,
      initial_conditions = init_conds
)

In [11]:
sim_model